In [ ]:
import matplotlib
%matplotlib inline

In [ ]:
import sys
import numpy as np
import math
from matplotlib import pyplot as plt
sys.path.append('D://measuring/')
from analysis.lib.fitting import fit,common


In [ ]:
from analysis.lib.fitting import fit,common
from analysis.lib.tools import plot
from analysis.scripts.laser_scan import plot_nv_levels
# from lifetime_purcell import Purcell_lifetime_reduction as Fp_t
from analysis.lib.tools import toolbox as tb

from analysis.lib.cavity import simulations_cavity

In [ ]:
#calculate cavity Finesse from length, linewidth
L =4.22e-6 #excluding diamond thickness - only air part
d_diamond=4.128e-6
dnu = 3.17e9

cs = simulations_cavity.CavitySims()
cs.set_cavity_length (L)
cs.set_diamond_thickness (d_diamond)


optical_length = cs.calc_optical_length(cavity_length=L, diamond_thickness=d_diamond)
nuFSR = cs.optical_length_to_nuFSR(optical_length)
F = cs.dnu_nuFSR_to_F(dnu, nuFSR)

print F


In [ ]:
c=2.99792e8
reload(simulations_cavity)

##################################################
####### create the cavity object
##################################################
cs = simulations_cavity.CavitySims()

##################################################
####### Define here your cavity parameters 
##################################################
L = 8.2e-6 #excluding diamond thickness - only air part
d_diamond=4.e-6
ROC = 20.e-6
wavelength = 637.e-9
nu = c/wavelength

cs.set_diamond_thickness (d_diamond)
cs.set_mirror_curvature (ROC)
cs.set_cavity_length (L)


##################################################
####### If you want to calculate paramters for a certain Q, fill in you favourite losses here.
##################################################
roughness1 = 0.3e-9
roughness2 = 0.3e-9
S1 = (4*math.pi*roughness1/wavelength)**2
T1 = 50e-6
A1 = 10e-6

S2 = (4*math.pi*roughness2/wavelength)**2
T2 = 50e-6
A2 = 10e-6

Ltot = S1+T1+A1+S2+T2+A2

nuFSR = cs.calc_FSR()

Finesse = cs.Ltot_to_F(Ltot)
Q = cs.F_to_Q(Finesse, nuFSR, nu)
dnu = cs.Q_nu_to_dnu(Q,nu)

cs.set_Q (Q/3)
# print 'scattering 1', S1*1e6, 'ppm'
# print 'scattering 2', S2*1e6, 'ppm'
print 'FSR', nuFSR/1.e12, 'THz'
# print 'nu', nu/1.e12, 'THz'
print 'finesse',Finesse
# print 'Q',Q/3
# print 'dnu (GHz)',dnu/1.e9

###############################################
######## calculate all derived parameters
###############################################
cs.calculate_params()

print 100*'*'
# print 'purcell factor',cs.calc_purcell_factor_air()
# print 'purcell factor',cs.calc_purcell_factor_diamond()

print 'diamond length',cs.get_diamond_thickness()*1.e6,'um'
print 'mirror radius of curvature', cs.get_mirror_curvature()*1.e6,'um'
print 'optical length',cs.get_optical_length()*1.e6,'um'
print 'mode volume ', cs.get_mode_volume()/1.e-18,'um^3'
print 100*'*'


###############################################
######## pick a Q and determine the corresponding maximum total losses
###############################################
Q = 60000.
Finesse = cs.Q_to_F(Q, nuFSR, nu)
Ltot = cs.F_to_Ltot(Finesse)

print 'Ltot <',round(Ltot*1.e6,1),'ppm needed for Q =', int(Q),'; Finesse =',round(Finesse,1)
print 'the cavity linewidth is then', cs.Q_nu_to_dnu(Q,nu)/1.e9 ,'GHz'
print 100*'*'


###############################################
######## plotting
###############################################

sweep_param1='Temperature (K)'
sweep_param2='Quality factor'
sweep_param3='dephasing rate (GHz)'
sweep_param4='cavity linewidth (GHz)'


#cs.plot_vs_sweepparam(sweep_param1, 0, 100, 100, plotmode='ZPL_PSB_inout')
#cs.plot_vs_sweepparam(sweep_param1, 0, 100, 100, plotmode='branching_ratio')
#cs.plot_vs_sweepparam(sweep_param1, 0, 100, 20, plotmode='rates')
# cs.plot_vs_sweepparam(sweep_param1, 0, 100, 20, plotmode='Fpurcell')
# cs.plot_vs_sweepparam(sweep_param1, 0, 100, 21, plotmode='ZPL_to_zero')
# cs.plot_vs_sweepparam(sweep_param1, 0, 100, 51, plotmode='lifetime', xlogscale = False)

cs.set_temperature (5) #after sweeping the temperature, set T to 5.
cs.plot_vs_sweepparam(sweep_param2, 0, 100000, 100, plotmode='branching_ratio')
# cs.plot_vs_sweepparam(sweep_param2, 44999, 45000, 3, plotmode='lifetime')
# cs.plot_vs_sweepparam(sweep_param2, 0, 100000, 100, plotmode='ZPL_PSB_inout')
cs.plot_vs_sweepparam(sweep_param2, 0, 400000, 100, plotmode='ZPL_to_zero')

cs.plot_vs_sweepparam(sweep_param2, 0, 40000, 500, plotmode='Fpurcell')
# cs.plot_vs_sweepparam(sweep_param3, 0.001, 1000, 500, plotmode='Fpurcell')


cs.plot_vs_sweepparam(sweep_param4, 1, 40, 100, plotmode='ZPL_PSB_inout')
cs.plot_vs_sweepparam(sweep_param4, 1, 40, 100, plotmode='ZPL_to_zero')

cs.plot_vs_sweepparam(sweep_param4, 1, 40, 500, plotmode='Fpurcell')





In [ ]:
execfile('../measuring/analysis/lib/cavity/simulations_cavity.py')

In [ ]:
c=3.e8
#freq = np.loadtxt('../Cavities/SpectrumData/X_data.txt')
counts = np.loadtxt('../Cavities/SpectrumData/Y_data_2_smooth.txt')
#wavelength  = c/(freq*1.e9) *1.e9
wavelength = np.loadtxt('../Cavities/SpectrumData/X_data_wl.txt')

wl = wavelength[425:]
cts = counts[425:]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(wl,cts,'darkslateblue',linewidth=2)

ax.set_xlabel('wavelength (nm)', fontsize = 18)
ax.set_ylabel('Intensity (a.u.)', fontsize = 18)
ax.xaxis.set_ticks(np.arange(630, 740, 30))
ax.yaxis.set_ticks(np.arange(600, 780, 150))
ax.tick_params(axis='both', which='major',   top='off',right = 'off',labelsize=14)
ax.set_xlim(620,735)
ax.set_ylim(590,760)
plt.tight_layout()


try:
    fig.savefig('H://My Documents/Cavities/SpectrumData/'+tb.get_timestamp_from_now()+'_NV_spectrum.pdf')
except: 
    print 'Figure not saved'


In [ ]:
def fit_3lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3):
    fitfunc_str = 'a1 + 2*A1/np.pi*gamma1/(4*(x-x01)**2+gamma1**2) \
            + 2*A2/np.pi*gamma2/(4*(x-x02)**2+gamma2**2)'

    a1 = fit.Parameter(g_a1, 'a1')
    
    A1 = fit.Parameter(g_A1, 'A1')
    x01 = fit.Parameter(g_x01, 'x01')
    gamma1 = fit.Parameter(g_gamma1, 'gamma1')

    A2 = fit.Parameter(g_A2, 'A2')
    x02 = fit.Parameter(g_x02, 'x02')
    gamma2 = fit.Parameter(g_gamma2, 'gamma2')

    A3 = fit.Parameter(g_A3, 'A3')
    x03 = fit.Parameter(g_x03, 'x03')
    gamma3 = fit.Parameter(g_gamma3, 'gamma3')
    
    p0 = [a1, A1, x01, gamma1, A2, x02, gamma2,A3, x03, gamma3]

    def fitfunc(x):
        return a1()+2*A1()/np.pi*gamma1()/(4*(x-x01())**2+gamma1()**2)+\
                2*A2()/np.pi*gamma2()/(4*(x-x02())**2+gamma2()**2)+\
                2*A3()/np.pi*gamma3()/(4*(x-x03())**2+gamma3()**2)

    return p0, fitfunc, fitfunc_str

def fit_4lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3, g_A4, g_x04, g_gamma4):
    fitfunc_str = 'a1 + 2*A1/np.pi*gamma1/(4*(x-x01)**2+gamma1**2) \
            + 2*A2/np.pi*gamma2/(4*(x-x02)**2+gamma2**2)\
            + 2*A3/np.pi*gamma3/(4*(x-x03)**2+gamma3**2)\
            + 2*A4/np.pi*gamma4/(4*(x-x04)**2+gamma4**2)'

    a1 = fit.Parameter(g_a1, 'a1')
    
    A1 = fit.Parameter(g_A1, 'A1')
    x01 = fit.Parameter(g_x01, 'x01')
    gamma1 = fit.Parameter(g_gamma1, 'gamma1')

    A2 = fit.Parameter(g_A2, 'A2')
    x02 = fit.Parameter(g_x02, 'x02')
    gamma2 = fit.Parameter(g_gamma2, 'gamma2')

    A3 = fit.Parameter(g_A3, 'A3')
    x03 = fit.Parameter(g_x03, 'x03')
    gamma3 = fit.Parameter(g_gamma3, 'gamma3')

    A4 = fit.Parameter(g_A4, 'A4')
    x04 = fit.Parameter(g_x04, 'x04')
    gamma4 = fit.Parameter(g_gamma4, 'gamma4')

    
    p0 = [a1, A1, x01, gamma1, A2, x02, gamma2,A3, x03, gamma3,A4,x04,gamma4]

    def fitfunc(x):
        return a1()+2*A1()/np.pi*gamma1()/(4*(x-x01())**2+gamma1()**2)+\
                2*A2()/np.pi*gamma2()/(4*(x-x02())**2+gamma2()**2)+\
                2*A3()/np.pi*gamma3()/(4*(x-x03())**2+gamma3()**2)+\
                2*A4()/np.pi*gamma4()/(4*(x-x04())**2+gamma4()**2)

    return p0, fitfunc, fitfunc_str

def fit_5lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3, g_A4, g_x04, g_gamma4,g_A5, g_x05, g_gamma5):
    fitfunc_str = 'a1 + 2*A1/np.pi*gamma1/(4*(x-x01)**2+gamma1**2) \
            + 2*A2/np.pi*gamma2/(4*(x-x02)**2+gamma2**2)\
            + 2*A3/np.pi*gamma3/(4*(x-x03)**2+gamma3**2)\
            + 2*A4/np.pi*gamma4/(4*(x-x04)**2+gamma4**2)\
            + 2*A5/np.pi*gamma5/(4*(x-x05)**2+gamma5**2)'

    a1 = fit.Parameter(g_a1, 'a1')
    
    A1 = fit.Parameter(g_A1, 'A1')
    x01 = fit.Parameter(g_x01, 'x01')
    gamma1 = fit.Parameter(g_gamma1, 'gamma1')

    A2 = fit.Parameter(g_A2, 'A2')
    x02 = fit.Parameter(g_x02, 'x02')
    gamma2 = fit.Parameter(g_gamma2, 'gamma2')

    A3 = fit.Parameter(g_A3, 'A3')
    x03 = fit.Parameter(g_x03, 'x03')
    gamma3 = fit.Parameter(g_gamma3, 'gamma3')

    A4 = fit.Parameter(g_A4, 'A4')
    x04 = fit.Parameter(g_x04, 'x04')
    gamma4 = fit.Parameter(g_gamma4, 'gamma4')

    A5 = fit.Parameter(g_A5, 'A5')
    x05 = fit.Parameter(g_x05, 'x05')
    gamma5 = fit.Parameter(g_gamma5, 'gamma5')

    
    p0 = [a1, A1, x01, gamma1, A2, x02, gamma2,A3, x03, gamma3,A4,x04,gamma4,A5,x05,gamma5]

    def fitfunc(x):
        return a1()+2*A1()/np.pi*gamma1()/(4*(x-x01())**2+gamma1()**2)+\
                2*A2()/np.pi*gamma2()/(4*(x-x02())**2+gamma2()**2)+\
                2*A3()/np.pi*gamma3()/(4*(x-x03())**2+gamma3()**2)+\
                2*A4()/np.pi*gamma4()/(4*(x-x04())**2+gamma4()**2)+\
                2*A5()/np.pi*gamma5()/(4*(x-x05())**2+gamma5()**2)

    return p0, fitfunc, fitfunc_str

def fit_6lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3, g_A4, g_x04, g_gamma4,g_A5, g_x05, g_gamma5,g_A6,g_x06,g_gamma6):
    fitfunc_str = 'a1 + 2*A1/np.pi*gamma1/(4*(x-x01)**2+gamma1**2) \
            + 2*A2/np.pi*gamma2/(4*(x-x02)**2+gamma2**2)\
            + 2*A3/np.pi*gamma3/(4*(x-x03)**2+gamma3**2)\
            + 2*A4/np.pi*gamma4/(4*(x-x04)**2+gamma4**2)\
            + 2*A5/np.pi*gamma5/(4*(x-x05)**2+gamma5**2)\
            + 2*A6/np.pi*gamma6/(4*(x-x06)**2+gamma6**2)'

    a1 = fit.Parameter(g_a1, 'a1')
    
    A1 = fit.Parameter(g_A1, 'A1')
    x01 = fit.Parameter(g_x01, 'x01')
    gamma1 = fit.Parameter(g_gamma1, 'gamma1')

    A2 = fit.Parameter(g_A2, 'A2')
    x02 = fit.Parameter(g_x02, 'x02')
    gamma2 = fit.Parameter(g_gamma2, 'gamma2')

    A3 = fit.Parameter(g_A3, 'A3')
    x03 = fit.Parameter(g_x03, 'x03')
    gamma3 = fit.Parameter(g_gamma3, 'gamma3')

    A4 = fit.Parameter(g_A4, 'A4')
    x04 = fit.Parameter(g_x04, 'x04')
    gamma4 = fit.Parameter(g_gamma4, 'gamma4')

    A5 = fit.Parameter(g_A5, 'A5')
    x05 = fit.Parameter(g_x05, 'x05')
    gamma5 = fit.Parameter(g_gamma5, 'gamma5')

    A6 = fit.Parameter(g_A6, 'A6')
    x06 = fit.Parameter(g_x06, 'x06')
    gamma6 = fit.Parameter(g_gamma6, 'gamma6')    
    
    p0 = [a1, A1, x01, gamma1, A2, x02, gamma2,A3, x03, gamma3,A4,x04,gamma4,A5,x05,gamma5,A6,x06,gamma6]

    def fitfunc(x):
        return a1()+2*A1()/np.pi*gamma1()/(4*(x-x01())**2+gamma1()**2)+\
                2*A2()/np.pi*gamma2()/(4*(x-x02())**2+gamma2()**2)+\
                2*A3()/np.pi*gamma3()/(4*(x-x03())**2+gamma3()**2)+\
                2*A4()/np.pi*gamma4()/(4*(x-x04())**2+gamma4()**2)+\
                2*A5()/np.pi*gamma5()/(4*(x-x05())**2+gamma5()**2)+\
                2*A6()/np.pi*gamma6()/(4*(x-x06())**2+gamma6()**2)

    return p0, fitfunc, fitfunc_str

In [ ]:

def delta_nu(gamma, x0):
    '''Function calculates the linewidth in frequency, from the linewidth in wavelength, and the center wavelength
    PARAMETERS:
    gamma - linewidth in wavelength in nm
    x0 - %%!linewidth center wavelength, in nm
    OUTPUT:
    delta_nu - linewidth in frequency, in THz
    '''
    delta_nu = c * (gamma*1.e-9) / ( (x0*1.e-9-(gamma*1.e-9/2.))*(x0*1.e-9+(gamma*1.e-9/2.) )) *1.e-12
    return delta_nu

def nu(x0):
    nu =  c/(x0*1.e-9) *1.e-12
    return nu


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(wl,cts)

g_a1 = 620
g_A1 = 17
g_x01 = 636.8
g_gamma1=0.06
g_A2 = 160
g_x02 = 660
g_gamma2 = 5
g_A3 = 110
g_x03 = 690
g_gamma3 = 10


# fixed =['g_A3','g_x03','g_gamma3']
fixed = []

p0, fitfunc, fitfunc_str = fit_3lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3)
fit_result = fit.fit1d(wl,cts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)

plot.plot_fit1d(fit_result, np.linspace(wl[0],wl[-1],10*len(wl)), add_txt=False, ax=ax,label='Fit',show_guess=False, plot_data=False)

print fit_result['params_dict']

a1 = fit_result['params_dict']['a1']
# a1 = g_a1
x01 = fit_result['params_dict']['x01']
A1 = fit_result['params_dict']['A1']
gamma1 = fit_result['params_dict']['gamma1']
x02 = fit_result['params_dict']['x02']
A2 = fit_result['params_dict']['A2']
gamma2 = fit_result['params_dict']['gamma2']
x03 = fit_result['params_dict']['x03']
A3 = fit_result['params_dict']['A3']
gamma3 = fit_result['params_dict']['gamma3']


y1 = a1 + 2*A1/np.pi*gamma1/(4*(wl-x01)**2+gamma1**2)
y2 = a1 + 2*A2/np.pi*gamma2/(4*(wl-x02)**2+gamma2**2)
y3 = a1 + 2*A3/np.pi*gamma3/(4*(wl-x03)**2+gamma3**2)
ax.plot(wl,y1)
ax.plot(wl,y2)
ax.plot(wl,y3)

ax.set_xlim(wl[0],wl[-1])
ax.set_ylim(610,770)
ax.set_xlabel('wavelength (nm)', fontsize = 16)
ax.set_ylabel('counts (a.u.)',fontsize = 16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.legend()

branching_ratio = A1 /(A1+A2+A3)
print 'branching_ratio',branching_ratio

In [ ]:
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(111)
ax.plot(wl,cts)

g_a1 = 630
g_A1 = 13
g_x01 = 636.8
g_gamma1=0.06
g_A2 = 190
g_x02 = 655
g_gamma2 = 3
g_A3 = 111
g_x03 = 690
g_gamma3 = 5
g_A4 = 50
g_x04 = 710
g_gamma4 = 1

# fixed =['g_A3','g_x03','g_gamma3']
fixed = []

p0, fitfunc, fitfunc_str = fit_4lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3,g_A4,g_x04,g_gamma4)
fit_result = fit.fit1d(wl,cts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)

plot.plot_fit1d(fit_result, np.linspace(wl[0],wl[-1],10*len(wl)), add_txt=False, ax=ax,label='Fit',show_guess=False, plot_data=False)

print fit_result['params_dict']

a1 = fit_result['params_dict']['a1']
# a1 = g_a1
x01 = fit_result['params_dict']['x01']
# x01=g_x01
A1 = fit_result['params_dict']['A1']
gamma1 = fit_result['params_dict']['gamma1']
x02 = fit_result['params_dict']['x02']
A2 = fit_result['params_dict']['A2']
gamma2 = fit_result['params_dict']['gamma2']
x03 = fit_result['params_dict']['x03']
A3 = fit_result['params_dict']['A3']
gamma3 = fit_result['params_dict']['gamma3']
x04 = fit_result['params_dict']['x04']
# x04 = g_x04
A4 = fit_result['params_dict']['A4']
# A4 = g_A4
gamma4 = fit_result['params_dict']['gamma4']
# gamma4 = g_gamma4

y1 = a1 + 2*A1/np.pi*gamma1/(4*(wl-x01)**2+gamma1**2)
y2 = a1 + 2*A2/np.pi*gamma2/(4*(wl-x02)**2+gamma2**2)
y3 = a1 + 2*A3/np.pi*gamma3/(4*(wl-x03)**2+gamma3**2)
y4 = a1 + 2*A4/np.pi*gamma4/(4*(wl-x04)**2+gamma4**2)

nu1 = nu(x01)
nu2 = nu(x02)
nu3 = nu(x03)
nu4 = nu(x04)

delta_nu1 = delta_nu(gamma1,x01)
delta_nu2 = delta_nu(gamma2,x02)
delta_nu3 = delta_nu(gamma3,x03)
delta_nu4 = delta_nu(gamma4,x04)

ax.plot(wl,y1,label='$\lambda_1$ = {0} nm (f = {4} THz), $\Delta\lambda_1$ = {2} nm ($\Delta f_1$ = {3} THz), A1 = {1}'.format(str(round(x01,1)),str(int(round(A1))),str(round(gamma1,1)),str(float(round(delta_nu1,1))),str(round(nu1,1))))
ax.plot(wl,y2,label='$\lambda_2$ = {0} nm (f = {4} THz), $\Delta\lambda_2$ = {2} nm ($\Delta f_2$ = {3} THz), A2 = {1}'.format(str(round(x02,1)),str(int(round(A2))),str(round(gamma2,1)),str(float(round(delta_nu2,1))),str(round(nu2,1))))
ax.plot(wl,y3,label='$\lambda_3$ = {0} nm (f = {4} THz), $\Delta\lambda_3$ = {2} nm ($\Delta f_3$ = {3} THz), A3 = {1}'.format(str(round(x03,1)),str(int(round(A3))),str(round(gamma3,1)),str(float(round(delta_nu3,1))),str(round(nu3,1))))
ax.plot(wl,y4,label='$\lambda_4$ = {0} nm (f = {4} THz), $\Delta\lambda_4$ = {2} nm ($\Delta f_4$ = {3} THz), A4 = {1}'.format(str(round(x04,1)),str(int(round(A4))),str(round(gamma4,1)),str(float(round(delta_nu4,1))),str(round(nu4,1))))


ax.set_xlim(wl[0],wl[-1])
ax.set_ylim(610,770)
ax.set_xlabel('wavelength (nm)', fontsize = 16)
ax.set_ylabel('counts (a.u.)',fontsize = 16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.legend()

branching_ratio = A1 / (A1+A2+A3+np.abs(A4))
print 'branching ratio',branching_ratio

try:
    fig.savefig('H://My Documents/Cavities/SpectrumData/'+tb.get_timestamp_from_now()+'fit_4lorentz.pdf')
except: 
    print 'Figure not saved'

In [ ]:
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(111)
ax.plot(wl,cts)

g_a1 = 630
g_A1 = 13
g_x01 = 636.8
g_gamma1=0.6
g_A2 = 366
g_x02 = 658
g_gamma2 = 10.7
g_A3 = 646
g_x03 = 680.8
g_gamma3 = 16.6
g_A4 = 439 #13
g_x04 = 693.8
g_gamma4 = 17
g_A5 = 488 #10
g_x05 = 708.6
g_gamma5 = 15.2


# fixed =['g_A3','g_x03','g_gamma3']
fixed = []

p0, fitfunc, fitfunc_str = fit_5lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3,g_A4,g_x04,g_gamma4,g_A5,g_x05,g_gamma5)
fit_result = fit.fit1d(wl,cts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)

plot.plot_fit1d(fit_result, np.linspace(wl[0],wl[-1],10*len(wl)), add_txt=False, ax=ax,label='Fit',show_guess=False, plot_data=False)

print fit_result['params_dict']

a1 = fit_result['params_dict']['a1']
# a1 = g_a1
x01 = fit_result['params_dict']['x01']
# x01=g_x01
A1 = fit_result['params_dict']['A1']
gamma1 = fit_result['params_dict']['gamma1']
x02 = fit_result['params_dict']['x02']
A2 = fit_result['params_dict']['A2']
gamma2 = fit_result['params_dict']['gamma2']
x03 = fit_result['params_dict']['x03']
A3 = fit_result['params_dict']['A3']
gamma3 = fit_result['params_dict']['gamma3']
x04 = fit_result['params_dict']['x04']
# x04 = g_x04
A4 = fit_result['params_dict']['A4']
# A4 = g_A4
gamma4 = fit_result['params_dict']['gamma4']
# gamma4 = g_gamma4
x05 = fit_result['params_dict']['x05']
# x05 = g_x05
A5 = fit_result['params_dict']['A5']
# A5 = g_A5
gamma5 = fit_result['params_dict']['gamma5']
# gamma5 = g_gamma5

print round(x01,1)
y1 = a1 + 2*A1/np.pi*gamma1/(4*(wl-x01)**2+gamma1**2)
y2 = a1 + 2*A2/np.pi*gamma2/(4*(wl-x02)**2+gamma2**2)
y3 = a1 + 2*A3/np.pi*gamma3/(4*(wl-x03)**2+gamma3**2)
y4 = a1 + 2*A4/np.pi*gamma4/(4*(wl-x04)**2+gamma4**2)
y5 = a1 + 2*A5/np.pi*gamma5/(4*(wl-x05)**2+gamma5**2)

nu1 = nu(x01)
nu2 = nu(x02)
nu3 = nu(x03)
nu4 = nu(x04)
nu5 = nu(x05)

delta_nu1 = delta_nu(gamma1,x01)
delta_nu2 = delta_nu(gamma2,x02)
delta_nu3 = delta_nu(gamma3,x03)
delta_nu4 = delta_nu(gamma4,x04)
delta_nu5 = delta_nu(gamma5,x05)

ax.plot(wl,y1,label='$\lambda_1$ = {0} nm (f = {4} THz), $\Delta\lambda_1$ = {2} nm ($\Delta f_1$ = {3} THz), A1 = {1}'.format(str(round(x01,1)),str(int(round(A1))),str(round(gamma1,1)),str(float(round(delta_nu1,1))),str(round(nu1,1))))
ax.plot(wl,y2,label='$\lambda_2$ = {0} nm (f = {4} THz), $\Delta\lambda_2$ = {2} nm ($\Delta f_2$ = {3} THz), A2 = {1}'.format(str(round(x02,1)),str(int(round(A2))),str(round(gamma2,1)),str(float(round(delta_nu2,1))),str(round(nu2,1))))
ax.plot(wl,y3,label='$\lambda_3$ = {0} nm (f = {4} THz), $\Delta\lambda_3$ = {2} nm ($\Delta f_3$ = {3} THz), A3 = {1}'.format(str(round(x03,1)),str(int(round(A3))),str(round(gamma3,1)),str(float(round(delta_nu3,1))),str(round(nu3,1))))
ax.plot(wl,y4,label='$\lambda_4$ = {0} nm (f = {4} THz), $\Delta\lambda_4$ = {2} nm ($\Delta f_4$ = {3} THz), A4 = {1}'.format(str(round(x04,1)),str(int(round(A4))),str(round(gamma4,1)),str(float(round(delta_nu4,1))),str(round(nu4,1))))
ax.plot(wl,y5,label='$\lambda_5$ = {0} nm (f = {4} THz), $\Delta\lambda_5$ = {2} nm ($\Delta f_5$ = {3} THz), A5 = {1}'.format(str(round(x05,1)),str(int(round(A5))),str(round(gamma5,1)),str(float(round(delta_nu5,1))),str(round(nu5,1))))

ax.set_xlim(wl[0],wl[-1])
ax.set_ylim(610,770)
ax.set_xlabel('wavelength (nm)', fontsize = 16)
ax.set_ylabel('counts (a.u.)',fontsize = 16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.legend()

lifetime  = 1./gamma1+1./gamma2+1./gamma3+1./gamma4+1./gamma5
print 'lifetime', lifetime
branching_ratio = A1 / (A1+A2+A3+np.abs(A4)+np.abs(A5))
print 'branching ratio',branching_ratio


try:
    fig.savefig('H://My Documents/Cavities/SpectrumData/'+tb.get_timestamp_from_now()+'fit_5lorentz.pdf')
except: 
    print 'Figure not saved'

In [ ]:
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(111)
ax.plot(wl,cts)

g_a1 = 612
g_A1 = 119
g_x01 = 636.8
g_gamma1=0.6
g_A2 = 820#4
g_x02 = 657.5#5
g_gamma2 = 16
g_A3 = 646
g_x03 = 680.8
g_gamma3 = 16.6
g_A4 = 439 #13
g_x04 = 694
g_gamma4 = 17
g_A5 = 488 #10
g_x05 = 709
g_gamma5 = 15
g_A6 = 400 #16
g_x06 = 660
g_gamma6 = 16

# fixed =['g_A3','g_x03','g_gamma3']
fixed = [5,17]

p0, fitfunc, fitfunc_str = fit_6lorentz(g_a1, g_A1, g_x01, g_gamma1, g_A2, g_x02, g_gamma2, g_A3, g_x03, g_gamma3,g_A4,g_x04,g_gamma4,g_A5,g_x05,g_gamma5,g_A6,g_x06,g_gamma6)
fit_result = fit.fit1d(wl,cts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)

plot.plot_fit1d(fit_result, np.linspace(wl[0],wl[-1],10*len(wl)), add_txt=False, ax=ax,label='Fit',show_guess=False, plot_data=False)

print fit_result['params_dict']

a1 = fit_result['params_dict']['a1']
# a1 = g_a1
x01 = fit_result['params_dict']['x01']
# x01=g_x01
A1 = fit_result['params_dict']['A1']
gamma1 = fit_result['params_dict']['gamma1']
# x02 = fit_result['params_dict']['x02']
x02=g_x02
A2 = fit_result['params_dict']['A2']
gamma2 = fit_result['params_dict']['gamma2']
x03 = fit_result['params_dict']['x03']
A3 = fit_result['params_dict']['A3']
gamma3 = fit_result['params_dict']['gamma3']
x04 = fit_result['params_dict']['x04']
# x04 = g_x04
A4 = fit_result['params_dict']['A4']
# A4 = g_A4
gamma4 = fit_result['params_dict']['gamma4']
# gamma4 = g_gamma4
x05 = fit_result['params_dict']['x05']
# x05 = g_x05
A5 = fit_result['params_dict']['A5']
# A5 = g_A5
gamma5 = fit_result['params_dict']['gamma5']
# gamma5 = g_gamma5
# x06 = fit_result['params_dict']['x06']
x06 = g_x06
A6 = fit_result['params_dict']['A6']
# A6 = g_A6
gamma6 = fit_result['params_dict']['gamma6']
# gamma6 = g_gamma6


y1 = a1 + 2*A1/np.pi*gamma1/(4*(wl-x01)**2+gamma1**2)
y2 = a1 + 2*A2/np.pi*gamma2/(4*(wl-x02)**2+gamma2**2)
y3 = a1 + 2*A3/np.pi*gamma3/(4*(wl-x03)**2+gamma3**2)
y4 = a1 + 2*A4/np.pi*gamma4/(4*(wl-x04)**2+gamma4**2)
y5 = a1 + 2*A5/np.pi*gamma5/(4*(wl-x05)**2+gamma5**2)
y6 = a1 + 2*A6/np.pi*gamma6/(4*(wl-x06)**2+gamma6**2)

def delta_nu(gamma, x0):
    '''Function calculates the linewidth in frequency, from the linewidth in wavelength, and the center wavelength
    PARAMETERS:
    gamma - linewidth in wavelength in nm
    x0 - %%!linewidth center wavelength, in nm
    OUTPUT:
    delta_nu - linewidth in frequency, in THz
    '''
    delta_nu = c * (gamma*1.e-9) / ( (x0*1.e-9-(gamma*1.e-9/2.))*(x0*1.e-9+(gamma*1.e-9/2.) )) *1.e-12
    return delta_nu
delta_nu1 = delta_nu(gamma1,x01)
delta_nu2 = delta_nu(gamma2,x02)
delta_nu3 = delta_nu(gamma3,x03)
delta_nu4 = delta_nu(gamma4,x04)
delta_nu5 = delta_nu(gamma5,x05)
delta_nu6 = delta_nu(gamma6,x06)


ax.plot(wl,y1,label='$\lambda_1$ = {0} nm, $\Delta\lambda_1$ = {2} nm ($\Delta f_1$ = {3} THz), A1 = {1}'.format(str(round(x01,1)),str(int(round(A1))),str(round(gamma1,1)),str(float(round(delta_nu1,2)))))
ax.plot(wl,y2,label='$\lambda_2$ = {0} nm, $\Delta\lambda_2$ = {2} nm ($\Delta f_2$ = {3} THz), A2 = {1}'.format(str(round(x02,1)),str(int(round(A2))),str(round(gamma2,1)),str(float(round(delta_nu2,2)))))
ax.plot(wl,y3,label='$\lambda_3$ = {0} nm, $\Delta\lambda_3$ = {2} nm ($\Delta f_3$ = {3} THz), A3 = {1}'.format(str(round(x03,1)),str(int(round(A3))),str(round(gamma3,1)),str(float(round(delta_nu3,2)))))
ax.plot(wl,y4,label='$\lambda_4$ = {0} nm, $\Delta\lambda_4$ = {2} nm ($\Delta f_4$ = {3} THz), A4 = {1}'.format(str(round(x04,1)),str(int(round(A4))),str(round(gamma4,1)),str(float(round(delta_nu4,2)))))
ax.plot(wl,y5,label='$\lambda_5$ = {0} nm, $\Delta\lambda_5$ = {2} nm ($\Delta f_5$ = {3} THz), A5 = {1}'.format(str(round(x05,1)),str(int(round(A5))),str(round(gamma5,1)),str(float(round(delta_nu5,2)))))
ax.plot(wl,y6,label='$\lambda_6$ = {0} nm, $\Delta\lambda_6$ = {2} nm ($\Delta f_6$ = {3} THz), A6 = {1}'.format(str(round(x06,1)),str(int(round(A6))),str(round(gamma6,1)),str(float(round(delta_nu6,2)))))


ax.set_xlim(wl[0],wl[-1])
ax.set_ylim(610,770)
ax.set_xlabel('wavelength (nm)', fontsize = 16)
ax.set_ylabel('counts (a.u.)',fontsize = 16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.legend()


#lifetime  = 1./gamma1+1./gamma3+1./gamma4+1./gamma5+1./gamma6
print 'lifetime', lifetime
branching_ratio = A1 / (A1+A2+A3+np.abs(A4)+np.abs(A5)+np.abs(A6))
print 'branching ratio',branching_ratio

In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_E_dependence()
# only E trasnsitions

In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_b_dependence()

In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_E_dependence()
# only A transitions

In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_E_dependence()
# only FB transitions (both A and E) and E' flip  (Ex<->Ey and vice versa)


In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_E_dependence(Bz = 30.)
# plot all ot forbidden or flip

In [ ]:
reload(plot_nv_levels)
plot_nv_levels.plot_transitions_E_dependence()
# plot all 

In [ ]:
reload(Fp_t)
Fp_t.check_NatComm_68206_2015()

